In [1]:
import numpy as np

In [2]:
def evaluation_forward(A, B, PI, O, alpha):
    alpha[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            alpha[j, t] = np.sum(np.multiply(alpha[:, t - 1], A[:, j])) * B[j, O[t]]
    return alpha

In [3]:
def evaluation_backward(A, B, PI, O, beta):
    beta[:, -1] = np.ones((A.shape[0]), dtype=np.float64)
    for t in range(O.shape[0] - 2, -1, -1):
        for i in range(A.shape[0]):
            beta[i, t] = np.sum(np.multiply(np.multiply(A[i, :], B[:, O[t]]), beta[:, t + 1]))
    return beta

In [4]:
def decode(A, B, PI, O, delta):
    delta[:, 0] = np.multiply(PI, B[:, O[0]])
    for t in range(1, O.shape[0]):
        for j in range(A.shape[0]):
            delta[j, t] = np.amax(np.multiply(delta[:, t - 1], A[:, j])) * B[j, O[t]]
    path = np.argmax(delta, axis=0) + 1
    return delta, path

In [5]:
def learn(A, B, PI, O, V, alpha, beta, XI, gamma, epsilon=10e-6):
    alpha = evaluation_forward(A, B, PI, O, alpha)
    beta = evaluation_backward(A, B, PI, O, beta)
    
    #Compute XI#
    p = np.sum(np.multiply(alpha, beta), axis=0)
    for t in range(O.shape[0] - 1):
        for i in range(A.shape[0]):
            for j in range(A.shape[0]):
                XI[i, j, t] = (alpha[i, t] * beta[j, t + 1] * A[i, j] * B[j, O[t + 1]]) / (p[t] + epsilon)
    
    #Compute gamma#
    for t in range(O.shape[0]):
        for i in range(A.shape[0]):
            gamma[i, t] = np.sum(XI[i, :, t])
            
    #Compute new A#
    for i in range(A.shape[0]):
        for j in range(B.shape[0]):
            A[i, j] = np.sum(XI[i, j, :]) / (np.sum(gamma[i]) + epsilon)

    #Compute new PI#
    for i in range(A.shape[0]):
        PI[i] = gamma[i, 0]
        
    #Compute new B#
    for j in range(A.shape[0]):
        for k in range(V.shape[0]):
            B[j, k] = np.sum(np.multiply(gamma[j], (O == V[k]).astype(np.float64))) / (np.sum(gamma[j]) + epsilon)
    return A, B, PI, gamma, XI, alpha, beta

In [6]:
A = np.array([ #1st Rank = From State, 2nd Rank = To State
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3],
    [1 / 3, 1 / 3, 1 / 3]
], dtype=np.float64)
PI = np.array([1/ 3, 1 / 3, 1 / 3], dtype=np.float64) #1st Rank = Init State
B = np.array([ #1st Rankg = State, 2nd Rank = Output Value
    [4 / 6, 2 / 6],
    [2 / 6, 4 / 6],
    [3 / 6, 3 / 6]
], dtype=np.float64)
O = np.array([0, 0, 1, 0, 1], dtype=np.int32) #0 = red, 1 = blue
V = np.array([0, 1], dtype=np.int32) #0 = red, 1 = blue
alpha = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
beta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
delta = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
gamma = np.zeros((B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = State, 2nd Rank = Time
XI = np.zeros((B.shape[0], B.shape[0], O.shape[0]), dtype=np.float64) #1st Rank = From State, 2nd Rank = To State, 3rd Rank = Time

In [7]:
records = list()
records.append((np.copy(A), np.copy(B), np.copy(PI)))
for i in range(100):
    A, B, PI, gamma, XI, alpha, beta = learn(A, B, PI, O, V, alpha, beta, XI, gamma)
    delta, path = decode(A, B, PI, O, delta)
    records.append((np.copy(A), np.copy(B), np.copy(PI)))
    print('Iters {}, Path: {}'.format(i + 1, path))

Iters 1, Path: [1 1 2 1 2]
Iters 2, Path: [1 1 2 1 2]
Iters 3, Path: [1 1 2 1 2]
Iters 4, Path: [1 1 2 1 2]
Iters 5, Path: [1 3 2 1 2]
Iters 6, Path: [1 1 2 1 2]
Iters 7, Path: [1 3 2 1 2]
Iters 8, Path: [1 1 2 1 2]
Iters 9, Path: [1 3 3 1 2]
Iters 10, Path: [1 1 3 1 3]
Iters 11, Path: [1 3 3 3 3]
Iters 12, Path: [1 3 3 1 3]
Iters 13, Path: [1 3 3 3 3]
Iters 14, Path: [1 3 3 1 3]
Iters 15, Path: [1 3 3 3 3]
Iters 16, Path: [1 1 3 1 3]
Iters 17, Path: [1 3 3 3 3]
Iters 18, Path: [1 1 3 1 3]
Iters 19, Path: [1 3 3 3 3]
Iters 20, Path: [1 1 3 1 3]
Iters 21, Path: [1 3 3 3 3]
Iters 22, Path: [1 1 3 1 3]
Iters 23, Path: [1 3 3 3 3]
Iters 24, Path: [1 1 3 1 3]
Iters 25, Path: [1 3 3 3 3]
Iters 26, Path: [1 1 3 1 3]
Iters 27, Path: [1 3 3 3 3]
Iters 28, Path: [1 1 3 1 3]
Iters 29, Path: [1 3 3 3 3]
Iters 30, Path: [1 1 3 1 3]
Iters 31, Path: [1 3 3 3 3]
Iters 32, Path: [1 1 3 1 3]
Iters 33, Path: [1 3 3 3 3]
Iters 34, Path: [1 1 3 1 3]
Iters 35, Path: [1 3 3 3 3]
Iters 36, Path: [1 1 3 1 3]
I

In [8]:
for i in range(97, 101):
    print('============ Iter {} ============'.format(i))
    print('A:\n{}'.format(records[i][0]))
    print('B:\n{}'.format(records[i][1]))
    print('PI:\n{}'.format(records[i][2]))

============ Iter 97 ============
A:
[[0.00386476 0.         0.99612597]
 [0.         0.         0.        ]
 [0.00115453 0.         0.99883538]]
B:
[[9.99990726e-001 2.26838440e-179]
 [0.00000000e+000 0.00000000e+000]
 [9.97807867e-001 2.18204587e-003]]
PI:
[5.25529716e-003 0.00000000e+000 9.24577721e-123]
============ Iter 98 ============
A:
[[0.0038731  0.         0.99610811]
 [0.         0.         0.        ]
 [0.0011547  0.         0.9988365 ]]
B:
[[9.99981212e-001 2.58574322e-179]
 [0.00000000e+000 0.00000000e+000]
 [2.03922359e-003 9.97951984e-001]]
PI:
[5.32239483e-001 0.00000000e+000 9.34332074e-121]
============ Iter 99 ============
A:
[[0.00380394 0.         0.99618617]
 [0.         0.         0.        ]
 [0.00109559 0.         0.99889416]]
B:
[[9.99990109e-001 3.47426052e-184]
 [0.00000000e+000 0.00000000e+000]
 [9.97849494e-001 2.14025453e-003]]
PI:
[4.93703901e-003 0.00000000e+000 8.02158570e-126]
============ Iter 100 ============
A:
[[0.00381199 0.         0.99616848]